In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import tsdm

In [ ]:
tsdm.datasets.MIMIC_III_RAW()

In [ ]:
import numpy as np
np.array(1)

In [ ]:
from math import prod

In [ ]:
x.ndim

In [ ]:
type(ds.ADMISSIONS.dtypes.iloc[10])

In [ ]:
import pandas as pd
pd.ArrowDtype

In [ ]:
import pyarrow as pa
pa.DataType

In [ ]:
repr(pa.float32())

In [ ]:
def repr_arrowdtype(dtype: pa.DataType, /) -> str:
    match dtype:
        case pa.DictionaryType(index_type=index_type, value_type=value_type):
            return f"dict[{index_type!s},{value_type!s}]"

In [ ]:
repr_arrowdtype(ds.ADMISSIONS.dtypes.iloc[10].pyarrow_dtype)

In [ ]:
str(pa.duration("s"))

In [ ]:
dir(ds.ADMISSIONS.dtypes.iloc[10].pyarrow_dtype)

In [ ]:
for name, frame in ds.items():
    print(f"{name:30s}, {sum(frame.memory_usage())}")

In [ ]:
import abc
from typing import Protocol, runtime_checkable


class MyMeta(type(Protocol)):
    
    
    def __subclasscheck__(self, other):
        print('in subclasscheck')
        return super().__subclasscheck__(other)

@runtime_checkable
class MyClsABC(Protocol, metaclass=MyMeta):
    
    def foo(self): ...
        
    
    @classmethod
    def __subclasshook__(cls, other):
        print('in subclasshook')
        print(super())
        return super().__subclasscheck__(other)

class SupportsFoo:
    def foo(self):
        pass
    
class SupportsBar:
    def bar(self):
        pass

In [ ]:
issubclass(SupportsFoo, MyClsABC)